# Numerične metode v tehniki
## Diferencialne enačbe - 1.del - IVP

- **avtor**: A.S. Grm
- **date**: 14/11/2023
- **primer**: 02

<hr>

## RK metoda 2. reda

S pomočjo RK metode 2. reda reši primer

\begin{align}
    y' -y & = 1 - x^2,\\
       & x \in [x_0,x_e], ~~~ x_0 = 0, x_e = \{2,4\}\\
       & y_0 = 0.5
\end{align}

Točna rešitev zgornjega IVP je

$$y = x^2 + 2 x - \frac{1}{2} e^x + 1$$



In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl

# MatPlotLib set fonts
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['DejaVu Serif']

# MatPlotLib set LaTeX use
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{siunitx}'

In [ ]:
# zapiši funkcijo odvoda v obliki python funkcije
def dydx(x,y):
    return y - x**2 + 1

# zapišimo člen RK metode 2. reda
def rk2(x,y,h):
    T2 = dydx(x + h/2,y + h/2*dydx(x,y))
    return T2

In [ ]:
# pridobi vsak n-ti element iz vektorja
def get_nth(x,n):
    if n == 0:
        return x
        
    v = x[0:-1:n]
    nn = v.size
    y = np.zeros(nn+1)
    y[0:nn] = v
    y[-1] = x[-1]
    return y

In [ ]:
def analytical(x):
    y = x**2 + 2*x - 1/2*np.exp(x) + 1
    return y

Izvedi glavni program za 

- $x_e=2$, $N=10$ 
- $x_e=4$, $N=10,20,40,80$

In [ ]:
# Glavni program
N = 10
x0 = 0
xe = 2
y0 = 0.5

# analitična rešitev
xa = np.linspace(x0,xe,100)
ya = analytical(xa)

# korak
h = (xe - x0)/N

# rezerviramo prostor za vektorje
x = np.zeros(N+1)
y = np.zeros(N+1)

# postavimo začetne vrednosti
x[0] = x0
y[0] = y0

# izvedemo izračun v obliki zanke
for i in range(N):
    # postavimo začetne vrednosti
    x_i = x0 + i*h
    y_old = y[i]

    # izračunamo y v novi točki
    y_new = y_old + h*rk2(x_i, y_old,h)

    # zapišemo rezultate v vektor
    x[i+1] = x0 + (i+1)*h
    y[i+1] = y_new

print('zadnja vrednost y({:d})={:.5f}'.format(N,y[N]))

In [ ]:
fig, ax = mpl.subplots()
fig.suptitle(r'Eulerjeva metoda -- Primer: $dy/dx = y - x^2 +1 $, $y_0=0.5$') # Figure title

ax.plot(xa,ya, 'b', label='exact')
ax.plot(x,y, 'g', label='implicite')
ax.plot(x,y, '.r')
ax.set_xlabel(r'$x_i$')
ax.set_ylabel(r'$y_i$')
ax.grid()

fig.tight_layout()
fig.savefig('01_primer-02.pdf')

## Določitev napake metode

In [ ]:
# izvedemo izračun v obliki zanke
Nv = np.array([10,20,40,80])
sNv = Nv.size
hv = np.zeros(Nv.size)

# inicializiramo vektorje za shranjevanje delnih rezultatov
xv = []
yv = []

for j in range(Nv.size):
    # inicializiramo stanje integracije
    h = (xe - x0)/Nv[j]
    hv[j] = h
    
    # rezerviramo prostor za zapis rezultatov
    x = np.zeros(Nv[j]+1)
    y = np.zeros(Nv[j]+1)
    y[0] = y0

    # izvedemo izračun
    for i in range(Nv[j]):
        # postavimo začetne vrednosti
        x_i = x0 + i*h
        y_old = y[i]
    
        # izračunamo y v novi točki
        y_new = y_old + h*rk2(x_i, y_old, h)
    
        # zapišemo rezultate v vektor
        x[i+1] = x0 + (i+1)*h
        y[i+1] = y_new

    xv.append(x)
    yv.append(y)

xa = xv[0]
ya = analytical(xa)
xv.append(xa)
yv.append(ya)

In [ ]:
# napako določimo kot povprečno na celem intervalu
errv = []
for i in range(4):
    if i == 0:
        k = 0
    else:
        k = 2**i
        
    errv.append(np.mean(np.abs(yv[-1] - get_nth(yv[i],k))))

In [ ]:
fig, ax = mpl.subplots()
fig.suptitle(r'Eulers method -- Error ($\Delta y_i = yi - y_{i+1}$)') # Figure title

ax.plot(hv,errv, '-g')
ax.plot(hv,errv, '.r')

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(r'step size $h$')
ax.set_ylabel(r'error $\Delta y$')
ax.grid()

fig.tight_layout()
fig.savefig('01_primer-02_error.pdf')

In [ ]:
# izračunam razlike v rešitvah v istih točkah, ki pomenijo napako
err = []
for i in range(4):
    if i == 0:
        k = 0
    else:
        k = 2**i
        
    err.append(yv[-1] - get_nth(yv[i],k))
    #err.append(np.divide(yv[-1] - get_nth(yv[i],k)),yv[-1]))

In [ ]:
fig, ax = mpl.subplots()
fig.suptitle(r'Eulers method -- Error ($\Delta y_i = yi - y_{i+1}$)') # Figure title

for i in range(Nv.size):
    ax.plot(xv[0],err[i], label='{:d}'.format(Nv[i]))

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(r'$x_i$')
ax.set_ylabel(r'error')
ax.legend()
ax.grid()